In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import sklearn.linear_model as sk_lin_mod
import math
import seaborn as sns
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy import stats
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression

import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib qt

In [2]:
os.getcwd()

print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
type(currDir)

donnee = pd.read_csv(fullPath)
stats=donnee.describe()
dimensions=donnee.shape
stats

/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/TP02


,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pixel6_B,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B
count,1875.000000,1875.000000,1873.00000,1875.000000,1873.000000,1875.000000,1872.000000,1875.000000,1875.000000,1875.000000,...,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000
mean,2010.000000,45.144000,1.51543,-8.633867,-3.557768,10.147680,2.738568,610.334347,20.633067,26.523733,...,33.771435,157.936992,71.317269,32.709104,156.869221,70.605899,33.460053,157.787557,70.609437,32.820418
std,6.059559,25.781811,7.63530,8.695859,7.843854,4.770659,5.574896,126.965290,5.251394,10.234991,...,26.416038,85.971146,52.296358,26.692262,85.231625,52.970868,27.297388,85.760582,53.761268,27.040843
min,2000.000000,1.000000,-22.60000,-34.800000,-25.700000,0.900000,0.000000,313.200000,14.000000,11.567238,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,23.000000,-3.60000,-15.100000,-9.300000,6.700000,0.000000,536.400000,16.000000,17.280176,...,8.345000,111.895000,32.555000,4.865000,111.960000,31.290000,5.545000,109.940000,28.830373,4.326591
50%,2010.000000,45.000000,1.70000,-6.700000,-2.500000,9.600000,0.000000,620.300000,20.000000,26.462002,...,32.970000,183.190000,71.920000,31.110000,180.810000,69.960000,31.410000,183.020000,70.558732,31.111611
75%,2015.000000,67.000000,6.40000,-1.450000,2.400000,13.100000,2.600000,685.100000,25.000000,34.315361,...,55.820000,227.420000,102.465000,54.910000,224.955000,103.015000,56.400000,225.820000,104.848430,55.027608
max,2020.000000,90.000000,27.50000,10.000000,18.800000,31.000000,41.600000,811.700000,30.000000,46.784424,...,108.800000,255.000000,255.000000,125.400000,255.000000,255.000000,117.940000,255.000000,255.000000,132.121595


In [3]:
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          2
Temp min.(°C)                          0
Temp moy.(°C)                          2
Diff Temp (°C)                         0
Précip. tot. (mm)                      3
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

Heureusement, très peu de nulls.

Quels sont les enregistrements contenant des nulls?


In [4]:
donnee[(donnee['Temp max.(°C)'].isna()==True) | (donnee['Précip. tot. (mm)'].isna()==True)] 

,Date,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,...,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B,Classe Sirop
1519,2017-02-02,2017,2,NaN,-17.4,NaN,17.4,NaN,770.6,20,...,224.35,173.08,30.45,255.00,127.83,77.18,255.00,127.017170,10.139073,Ambré
1524,2017-02-07,2017,7,NaN,-25.1,NaN,25.1,NaN,770.6,20,...,255.00,131.77,3.27,225.25,123.51,61.07,194.07,70.795317,71.603721,Foncé
1632,2018-02-26,2018,26,3.2,-7.4,-2.1,10.6,NaN,811.7,16,...,213.40,111.76,3.39,212.26,98.49,35.60,193.85,127.018875,37.339503,Foncé


Fill na

In [5]:
donnee.fillna(method='bfill', inplace=True)
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

In [6]:
donnee.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'Pixel1_R', 'Pixel1_G', 'Pixel1_B', 'Pixel2_R', 'Pixel2_G', 'Pixel2_B',
       'Pixel3_R', 'Pixel3_G', 'Pixel3_B', 'Pixel4_R', 'Pixel4_G', 'Pixel4_B',
       'Pixel5_R', 'Pixel5_G', 'Pixel5_B', 'Pixel6_R', 'Pixel6_G', 'Pixel6_B',
       'Pixel7_R', 'Pixel7_G', 'Pixel7_B', 'Pixel8_R', 'Pixel8_G', 'Pixel8_B',
       'Pixel9_R', 'Pixel9_G', 'Pixel9_B', 'Classe Sirop'],
      dtype='object')

Supprimer les colonnes pixel

In [7]:
donnee.filter(like='Pixel').columns
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1).copy()
dfSansPixel.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'Classe Sirop'],
      dtype='object')

In [8]:
dfSansPixel.dtypes

Date                                    object
Année                                    int64
Jour Calendrier Saison                   int64
Temp max.(°C)                          float64
Temp min.(°C)                          float64
Temp moy.(°C)                          float64
Diff Temp (°C)                         float64
Précip. tot. (mm)                      float64
Précip. Tot. Hiver (mm)                float64
Nombre épisodes gel/dégel                int64
Production moyenne par entaille (L)    float64
Débit sève (L/j)                       float64
Sucre sève (%)                         float64
Alimentation osmoseur (L/j)            float64
Osmoseur (heures opération/j)          float64
Pression osmoseur (bar)                float64
Sucre sortie osmoseur (%)              float64
Température Bouilloire (0C)            float64
Temps bouilloire (h)                   float64
Sucre du sirop obtenu (%)                int64
Quantité de sirop obtenue (L)          float64
Transmittance

In [9]:
dfSansPixel.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Classe Sirop                           0
dtype: int64

In [10]:
dfSansPixel.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.51,-8.63,-3.57,10.15,2.74,610.33,20.63,26.52,...,2.06,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,0.94,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,2.00,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,2.34,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,2.67,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [11]:
### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]

## convertir Classe Sirop en numérique - https://www.youtube.com/watch?v=wH_ezgftiy0&t=136s
#dfSansPixel['CategClasseSirop'] = dfSansPixel['Classe Sirop'].astype('category').cat.codes
dfSansPixel = dfSansPixel.loc[dfSansPixel['Débit sève (L/j)'] !=0]
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,...,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00
mean,2010.32,50.42,2.77,-7.16,-2.20,9.93,2.89,620.42,20.80,26.99,...,2.42,1220.75,2.75,40.04,8.04,104.05,2.37,67.03,87.80,43.58
std,5.99,23.96,7.30,8.08,7.34,4.69,5.88,126.41,5.15,10.06,...,0.42,1832.68,4.12,2.02,2.02,0.20,1.62,1.02,133.13,11.70
min,2000.00,1.00,-20.00,-32.50,-24.00,0.90,0.00,313.20,14.00,11.57,...,0.08,19.00,0.04,33.40,1.40,103.28,1.50,63.00,0.38,6.79
25%,2005.00,31.00,-1.80,-12.40,-7.20,6.60,0.00,536.40,16.00,17.28,...,2.20,184.00,0.41,38.71,6.71,103.88,1.57,66.00,12.78,35.88
50%,2010.00,52.00,3.00,-5.10,-0.70,9.30,0.00,631.10,20.00,26.72,...,2.43,329.00,0.74,40.02,8.02,104.05,1.64,67.00,23.72,43.18
75%,2016.00,71.00,7.20,-0.90,3.00,12.90,2.60,688.30,25.00,35.57,...,2.72,1429.00,3.22,41.40,9.40,104.22,2.02,68.00,104.27,51.46
max,2020.00,90.00,27.50,10.00,18.80,27.50,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [12]:
dfNumerique.isnull().sum()

Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
dtype: int64

In [13]:
dfNumerique.shape

(1595, 21)

In [14]:
dfNumerique.loc[dfNumerique['Débit sève (L/j)'] ==0].count()

Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
dtype: int64

In [15]:
#### enelever les données où débit nul de sève
#dfNumerique = dfNumerique.loc[dfNumerique['Pression osmoseur (bar)'] !=0]
#dfNumerique = dfNumerique.loc[dfNumerique['Débit sève (L/j)'] !=0]
dfNumerique.loc[dfNumerique['Débit sève (L/j)'] ==0].count()

Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
dtype: int64

In [16]:
### convertir Classe Sirop en var numérique
###
# Creating numeric columns
###
colsCategSirop = pd.get_dummies(dfSansPixel['Classe Sirop'], prefix='categSirop_', drop_first=True)

## concatenate side by side
dfNumerique = pd.concat([dfNumerique, colsCategSirop], axis=1)
#dfNumeriue.columns


In [17]:
dfNumerique.shape

(1595, 24)

In [18]:
dfNumerique[
    #(dfNumerique['categSirop__Ambré']==0) &
    (dfNumerique['categSirop__Doré']==0) &
    (dfNumerique['categSirop__Foncé']==0) &
    (dfNumerique['categSirop__Très Foncé']==0)
]

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%),categSirop__Doré,categSirop__Foncé,categSirop__Très Foncé
23,2000,24,5.7,-3.1,1.3,8.8,0.0,313.2,27,34.315361,...,40.20,8.20,103.874690,4.27,66,328.89,52.55,0,0,0
28,2000,29,5.4,-2.8,1.3,8.2,0.0,313.2,27,34.315361,...,42.48,10.48,103.842770,3.63,66,299.90,55.46,0,0,0
30,2000,31,1.4,-0.1,0.7,1.5,4.6,313.2,27,34.315361,...,44.09,12.09,104.041176,1.54,67,5.13,54.08,0,0,0
32,2000,33,2.6,-2.8,-0.1,5.4,0.0,313.2,27,34.315361,...,40.82,8.82,104.064598,1.64,67,19.93,57.19,0,0,0
33,2000,34,5.0,-1.8,1.6,6.8,0.4,313.2,27,34.315361,...,38.80,6.80,104.211629,3.41,68,236.99,53.78,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,2020,63,8.0,1.0,4.5,7.0,0.0,688.3,30,46.784424,...,41.34,9.34,103.898194,1.68,66,24.71,54.24,0,0,0
1849,2020,65,7.9,-0.8,3.6,8.7,0.7,688.3,30,46.784424,...,41.50,9.50,104.014461,3.47,67,289.37,57.75,0,0,0
1857,2020,73,10.8,3.8,7.3,7.0,37.1,688.3,30,46.784424,...,40.89,8.89,104.279219,1.63,68,8.90,54.86,0,0,0
1858,2020,74,5.7,-0.8,2.4,6.5,0.0,688.3,30,46.784424,...,39.73,7.73,104.073306,2.56,67,214.19,60.54,0,0,0


In [19]:
### Standardization
## https://datagy.io/pandas-normalize-column/
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dfNumerique)
scaled = scaler.fit_transform(dfNumerique)
dfNumeriqueStd = pd.DataFrame(scaled, columns=dfNumerique.columns)

In [20]:
## book Practical Statiscs for Data Scientists, cahp 2
## https://github.com/gedeck/practical-statistics-for-data-scientists/blob/master/python/notebooks/Chapter%202%20-%20Data%20and%20sampling%20distributions.ipynb
def plot_prob_var_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
        #print("col[y]", dfNumerique[y][dfNumerique[y]>0])
        np.diff(np.log(col_y))
    
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_title( y)
        sts.probplot(col_y, plot=ax)

In [21]:
#plot_distr_var_y(dfNumerique)

In [22]:
#plot_distr_var_y(dfNumeriqueStd)

In [23]:
dfNumerique.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%),categSirop__Doré,categSirop__Foncé,categSirop__Très Foncé
count,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,...,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00
mean,2010.32,50.42,2.77,-7.16,-2.20,9.93,2.89,620.42,20.80,26.99,...,40.04,8.04,104.05,2.37,67.03,87.80,43.58,0.01,0.67,0.05
std,5.99,23.96,7.30,8.08,7.34,4.69,5.88,126.41,5.15,10.06,...,2.02,2.02,0.20,1.62,1.02,133.13,11.70,0.09,0.47,0.22
min,2000.00,1.00,-20.00,-32.50,-24.00,0.90,0.00,313.20,14.00,11.57,...,33.40,1.40,103.28,1.50,63.00,0.38,6.79,0.00,0.00,0.00
25%,2005.00,31.00,-1.80,-12.40,-7.20,6.60,0.00,536.40,16.00,17.28,...,38.71,6.71,103.88,1.57,66.00,12.78,35.88,0.00,0.00,0.00
50%,2010.00,52.00,3.00,-5.10,-0.70,9.30,0.00,631.10,20.00,26.72,...,40.02,8.02,104.05,1.64,67.00,23.72,43.18,0.00,1.00,0.00
75%,2016.00,71.00,7.20,-0.90,3.00,12.90,2.60,688.30,25.00,35.57,...,41.40,9.40,104.22,2.02,68.00,104.27,51.46,0.00,1.00,0.00
max,2020.00,90.00,27.50,10.00,18.80,27.50,41.60,811.70,30.00,46.78,...,46.59,14.59,104.69,11.48,70.00,681.99,82.52,1.00,1.00,1.00


In [24]:
dfNumeriqueStd.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%),categSirop__Doré,categSirop__Foncé,categSirop__Très Foncé
count,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,...,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00,1595.00
mean,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,...,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-1.73,-2.06,-3.12,-3.14,-2.97,-1.92,-0.49,-2.43,-1.32,-1.53,...,-3.28,-3.28,-3.80,-0.54,-3.95,-0.66,-3.14,-0.09,-1.42,-0.23
25%,-0.89,-0.81,-0.63,-0.65,-0.68,-0.71,-0.49,-0.66,-0.93,-0.97,...,-0.66,-0.66,-0.84,-0.50,-1.01,-0.56,-0.66,-0.09,-1.42,-0.23
50%,-0.05,0.07,0.03,0.25,0.20,-0.13,-0.49,0.08,-0.16,-0.03,...,-0.01,-0.01,0.00,-0.45,-0.03,-0.48,-0.03,-0.09,0.71,-0.23
75%,0.95,0.86,0.61,0.78,0.71,0.63,-0.05,0.54,0.82,0.85,...,0.67,0.67,0.83,-0.22,0.96,0.12,0.67,-0.09,0.71,-0.23
max,1.62,1.65,3.39,2.13,2.86,3.74,6.59,1.51,1.79,1.97,...,3.24,3.24,3.12,5.61,2.92,4.46,3.33,11.49,0.71,4.35


In [25]:
def plot_box_plot_vars_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
    
        plt.figure()
        sns.boxplot(y=col_y).set_title(y)
        plt.show(block=False)

In [26]:
#plot_box_plot_vars_y(dfNumerique)

In [27]:
len(dfNumerique)

1595

### Multiple Linear Regression


https://www.geeksforgeeks.org/ml-multiple-linear-regression-using-python/

Assumption of Regression Model : 

- Linearity: The relationship between dependent and independent variables should be linear.
- Homoscedasticity: Constant variance of the errors should be maintained.
- Multivariate normality: Multiple Regression assumes that the residuals are normally distributed.
- Lack of Multicollinearity: It is assumed that there is little or no multicollinearity in

In [28]:
stats = dfNumerique.describe()
stats

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%),categSirop__Doré,categSirop__Foncé,categSirop__Très Foncé
count,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,...,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000
mean,2010.324138,50.418182,2.766144,-7.158558,-2.195549,9.931850,2.885016,620.421755,20.799373,26.989449,...,40.037329,8.037329,104.054336,2.374075,67.025705,87.801266,43.584125,0.007524,0.667712,0.050157
std,5.986152,23.964451,7.295008,8.077084,7.344902,4.694653,5.879401,126.414498,5.145182,10.063518,...,2.021187,2.021187,0.204810,1.622381,1.018630,133.134084,11.704837,0.086438,0.471182,0.218337
min,2000.000000,1.000000,-20.000000,-32.500000,-24.000000,0.900000,0.000000,313.200000,14.000000,11.567238,...,33.400000,1.400000,103.277229,1.500000,63.000000,0.380000,6.790000,0.000000,0.000000,0.000000
25%,2005.000000,31.000000,-1.800000,-12.400000,-7.200000,6.600000,0.000000,536.400000,16.000000,17.280176,...,38.710000,6.710000,103.882920,1.570000,66.000000,12.780000,35.875000,0.000000,0.000000,0.000000
50%,2010.000000,52.000000,3.000000,-5.100000,-0.700000,9.300000,0.000000,631.100000,20.000000,26.721388,...,40.020000,8.020000,104.054607,1.640000,67.000000,23.720000,43.180000,0.000000,1.000000,0.000000
75%,2016.000000,71.000000,7.200000,-0.900000,3.000000,12.900000,2.600000,688.300000,25.000000,35.569783,...,41.395000,9.395000,104.224430,2.020000,68.000000,104.270000,51.460000,0.000000,1.000000,0.000000
max,2020.000000,90.000000,27.500000,10.000000,18.800000,27.500000,41.600000,811.700000,30.000000,46.784424,...,46.590000,14.590000,104.694042,11.480000,70.000000,681.990000,82.520000,1.000000,1.000000,1.000000


### Séparation des variables dépendantes (Y) et indépendantes (X)

In [29]:
def get_dfX_dfY(pDfToSplit, pListDependentVars):
    ## seulement les variables X
    df_X_cols = pDfToSplit.loc[:, ~pDfToSplit.columns.isin(pListDependentVars)].copy()


    ## seulement les variables Y
    df_Y_cols = pDfToSplit.loc[:, pDfToSplit.columns.isin(pListDependentVars)].copy()
    
    return df_X_cols, df_Y_cols


In [30]:
## Obtentions de variables X et Y dans des df séparés


dfNumerique_X_cols, dfNumerique_Y_cols = get_dfX_dfY(dfNumerique, list_cols_dependennt_vars)


In [31]:
### imprimer matrice correlation
def plot_correl(pDfCorrel):
    %matplotlib qt5    
    sns.heatmap(pDfCorrel, cbar=True, cmap="Blues", center=0, annot=True, fmt=".2f")

In [32]:
### imprimer matrice correlation
plot_correl(dfNumerique_X_cols.corr())

In [33]:
### identifier hautes correlation (>0.6)
#### On va eliminer les variables avec haute correlation
def print_high_correl(pDf, threshold=0.6):
    corr = pDf.corr()
    x = corr.unstack().sort_values(kind="quicksort", ascending=False).apply(lambda  x: str(x).split())
    type(x)
    ## reset column names
    x = pd.DataFrame(x).reset_index()
    x.columns = ['col1', 'col2', 'corr']
    ## transform corr from [1] to 1
    x['corr'] = x['corr'].apply(lambda x: float(np.squeeze(x)))

    ## paires de colonnes avec des corrélation élevées (> 0.6)
    print(x[(x['corr']>0.6) & (x['col1']!=x['col2'])].sort_values('col1'))

In [34]:
### imprimer les correlations
print_high_correl(dfNumerique_X_cols)

                             col1                           col2      corr
36    Alimentation osmoseur (L/j)           Temps bouilloire (h)  0.907063
22    Alimentation osmoseur (L/j)  Osmoseur (heures opération/j)  1.000000
31    Alimentation osmoseur (L/j)  Quantité de sirop obtenue (L)  0.954982
43                          Année        Précip. Tot. Hiver (mm)  0.770270
48         Jour Calendrier Saison                  Temp min.(°C)  0.678497
45         Jour Calendrier Saison                  Temp moy.(°C)  0.737940
46         Jour Calendrier Saison                  Temp max.(°C)  0.732056
33  Osmoseur (heures opération/j)  Quantité de sirop obtenue (L)  0.954974
39  Osmoseur (heures opération/j)           Temps bouilloire (h)  0.907045
23  Osmoseur (heures opération/j)    Alimentation osmoseur (L/j)  1.000000
20        Pression osmoseur (bar)      Sucre sortie osmoseur (%)  1.000000
42        Précip. Tot. Hiver (mm)                          Année  0.770270
30  Quantité de sirop obt

In [35]:
### supprime une liste de colonnes d'un dataframe
def delete_columns_df(pDf, pListeExclusion):
    pDf = pDf.loc[:, ~pDf.columns.isin([col for col in pListeExclusion])]
    return pDf

In [36]:
#### On drope des colonnes avec des correlation presquer parfaite
#lstColTemp = ['Temp max.(°C)', 'Temp min.(°C)']
#dfNumerique_X_cols = dfNumerique_X_cols.loc[:,~dfNumerique_X_cols.columns.isin(lstColTemp) ].copy()


lstColsExlude = ['Précip. Tot. Hiver (mm)', 'Temp moy.(°C)'
                 , 'Osmoseur (heures opération/j)'
                 , 'Alimentation osmoseur (L/j)'
                 , 'Temps bouilloire (h)' 
                 , 'Sucre sortie osmoseur (%)'
                 , 'Température Bouilloire (0C)'
                 , 'Temp min.(°C)'
                 , 'Temp max.(°C)'
                ]
#dfNumerique_X_cols = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin([col for col in lstColsExlude])]
dfNumerique_X_cols = delete_columns_df(dfNumerique_X_cols, lstColsExlude)


In [37]:
### on revoit les correlation
print_high_correl(dfNumerique_X_cols)

Empty DataFrame
Columns: [col1, col2, corr]
Index: []


In [38]:
## on plot la matrice de nouveau
plot_correl(dfNumerique_X_cols.corr())

In [39]:
plot_correl(dfNumerique.corr())

In [40]:
### Fonction qui imprime le sommaire du modèle
def print_model_summary(pDfColsX, pDfColsY):
    ## Add constant
    pDfColsX = stm.add_constant(pDfColsX)
    lst_models = []
    for y in (pDfColsY.columns.values):
    
        col_y = dfNumerique_Y_cols[y]
    
        model = stm.OLS(col_y, pDfColsX.assign(const=1)).fit()
        print("=========")
        print("var depend: ", y)
    
        #lst_models.append(model)
        print(model.summary())
        #sns.distplot(model.resid, fit=sts.norm)        
        #p = sns.histplot(model.resid, kde=True, stat="density").set(title = y)
        

In [41]:
print("---- Sommaire dfNumerique ----")
print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

---- Sommaire dfNumerique ----
var depend:  Production moyenne par entaille (L)
                                     OLS Regression Results                                    
Dep. Variable:     Production moyenne par entaille (L)   R-squared:                       0.942
Model:                                             OLS   Adj. R-squared:                  0.941
Method:                                  Least Squares   F-statistic:                     2330.
Date:                                 Wed, 14 Dec 2022   Prob (F-statistic):               0.00
Time:                                         18:54:45   Log-Likelihood:                -3677.0
No. Observations:                                 1595   AIC:                             7378.
Df Residuals:                                     1583   BIC:                             7442.
Df Model:                                           11                                         
Covariance Type:                             nonrobust  

In [42]:
### Traitement de la haute colinéarité (VIF)
### vif
## https://www.kdnuggets.com/2019/07/check-quality-regression-model-python.html
### https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/
#from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [43]:
### imprimer les colonnes avec haute colinéarité
#def print_calculate_vif(pDf):
#    # VIF dataframe
#    vif_data = pd.DataFrame()
#    vif_data["feature"] = pDf.columns
#    # calculating VIF for each feature
#    vif_data["VIF"] = [vif(dfNumerique_X_cols.values, i)
#                       for i in range(len(dfNumerique_X_cols.columns))]
#    print(vif_data.sort_values(by="VIF", axis=0, kind="quicksort",  ascending=False)[vif_data["VIF"]>10])

In [44]:
#print_calculate_vif(dfNumerique_X_cols)

In [45]:
#lstColsExlude=["Année"
               #, "Sucre du sirop obtenu (%)"
              # , "Pression osmoseur (bar)"
              # , "Nombre épisodes gel/dégel"
              #]
#dfNumerique_X_cols = delete_columns_df(dfNumerique_X_cols, lstColsExlude)

In [46]:
#print("---- Sommaire dfNumerique ----")
#print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

In [47]:

#### RÉPONSES AUX QUESTIONS Q1


### Les tableaux ci-bas donnent, entre autres les informations concernant le adj. r-squared
###, t-value, p-value
### et coefficients des modèles.
### !!!NOTE!!! générer les graphiques et le sauvegarder à part!!!

In [48]:
### https://github.com/simaria22/prediction_heart_failure/blob/master/feature_select.py
## Ajoute des colonnes au modèle pendant que la nouvelle colonne améliore
## le modèle OU
## jusqu'à qu'un certain niveau de précision soit atteint
def get_reduced_df_X(pDfx, pY, pAccuracy=0.7):
    print("------")
    print("dependent var y: ", pY.name)
    
    # new X df with fewer columns than pDfx
    reducedDf = []

    # column names to keep
    dfKeeper = None

    #no of features
    nof_list=np.arange(1,len(pDfx.columns))            
    high_score=0
    #Variable to store the optimum features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(pDfx, pY, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model,n_features_to_select= nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        
        print("n: ", n)
        print("score: ", score)
        print("high_score: ", high_score)

        # print summaries for the selection of attributes
        print("rfe.support_: ",rfe.support_)
        print("rfe.ranking_: " ,rfe.ranking_)
        
        #dfKeeper = pd.DataFrame(rfe.support_,index=pDfx.columns,columns=['Rank'])
        data= {'colName': pDfx.columns, 'keep': rfe.support_ }
        dfKeeper = pd.DataFrame(data)
        dfKeeper = dfKeeper[dfKeeper["keep"]==True]
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
        if (score > pAccuracy):
            break

    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("df columns to keep: ")
    #print("shape")
    #print(dfNumerique_X_cols[dfKeeper["colName"]].shape)
    print(dfKeeper)
    
    reducedDf = pDfx[dfKeeper["colName"]]
    
    #print("reducedDf: ", reducedDf.head())
    #print("unique: ", reducedDf[dfKeeper["colName"]].unique())
    return reducedDf

In [49]:
### création d'un dictionnaire contenant la variable dépendantes et ses variables indépendantes
### correspondantes
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence
dict_y_colsX = {}

dfNumerique_X_cols_orig, dfNumerique_Y_cols_orig = get_dfX_dfY(dfNumerique, list_cols_dependennt_vars)

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    #reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    reducedDf = get_reduced_df_X(dfNumerique_X_cols_orig, col_y)
    
    #print("reduceDf.columns.values: ",  reducedDf.columns.values)
    #print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    #print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    #print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    dict_y_colsX.update({col_y.name: reducedDf})

------
dependent var y:  Production moyenne par entaille (L)
n:  0
score:  0.9152801150789349
high_score:  0
rfe.support_:  [False False False False False False False False  True False False False
 False False False False False False False False]
rfe.ranking_:  [ 5 20  3 19  2  4 17 15  1 13 11 14 16  6  9  7 18 10 12  8]
Optimum number of features: 1
Score with 1 features: 0.915280
df columns to keep: 
                     colName  keep
8  Nombre épisodes gel/dégel  True
------
dependent var y:  Débit sève (L/j)
n:  0
score:  0.9463987440410185
high_score:  0
rfe.support_:  [False False False False False False False False False False False False
 False False  True False False False False False]
rfe.ranking_:  [18 19  7 11  8  9 13 20 14 16  2 12 17  5  1  6 15 10  4  3]
Optimum number of features: 1
Score with 1 features: 0.946399
df columns to keep: 
                 colName  keep
14  Temps bouilloire (h)  True
------
dependent var y:  Sucre sève (%)
n:  0
score:  0.44425353039029414

In [56]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("[key]: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())

[key]:  Production moyenne par entaille (L)
var depend:  Production moyenne par entaille (L)
                                     OLS Regression Results                                    
Dep. Variable:     Production moyenne par entaille (L)   R-squared:                       0.910
Model:                                             OLS   Adj. R-squared:                  0.909
Method:                                  Least Squares   F-statistic:                 1.602e+04
Date:                                 Wed, 14 Dec 2022   Prob (F-statistic):               0.00
Time:                                         19:01:53   Log-Likelihood:                -4029.1
No. Observations:                                 1595   AIC:                             8062.
Df Residuals:                                     1593   BIC:                             8073.
Df Model:                                            1                                         
Covariance Type:                           

-----

À ce point, on a un modèle avec une bonne précision (R-squared et Adj R-squared) pour 
toutes les variables dépendantes.


-----

In [57]:
### https://www.youtube.com/watch?v=VCVhwjbI6h8
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

def analyze_model(pColsY, pDfDependentVars):
    for y in (pColsY.columns.values):    
        col_y = pColsY[y]
        X_train, X_test, y_train, y_test = \
        train_test_split(pDfDependentVars, col_y, test_size=0.25, random_state=0)


        ## transforming data    
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)


        ## Fitting Multiple Linear Regression to the training set
        regressor =  LinearRegression()
        regressor.fit(X_train, y_train)
        

        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = math.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        #r_sq = regressor.score(X_test, y_test)

        ##adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
        ## Where n is the sample size and p is the number of independent variables.

        #print ("len(X_test): ", len(X_test))
        #print ("len(y_test): ", len(y_test))
    
        ### ajd_r2
        ## https://stackoverflow.com/questions/51038820/how-to-calculated-the-adjusted-r2-value-using-scikit
        ## https://www.dummies.com/article/business-careers-money/business/accounting/calculation-analysis/how-to-calculate-the-adjusted-coefficient-of-determination-146054/
        #print ("n: ", n)
        #print ("p: ", p)
        #print ("adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)")

        n = len(X_test)
        p = len(pDfDependentVars.columns)
        adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    
        print("--------")
        
        print("[col_y]: ", col_y.name)
        print("[mse]: ", mse)
        print("[rmse]: ", rmse)
        #print("[r2] - A value of 1 indicates that the response variable can be perfectly explained by the predictor variables.")
        print("[r2]: ", r2)
        print("[adj_r2]: ", adj_r2)
        print("[coef]: ", regressor.coef_)
        print("[intercept_]: ", regressor.intercept_)
        #print("[r_sq]: ", r_sq)

In [52]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]
    
    analyze_model(dfY.to_frame(), dfX)

key:  Production moyenne par entaille (L)
--------
[col_y]:  Production moyenne par entaille (L)
[mse]:  8.73225715053533
[rmse]:  2.9550392807093666
[r2]:  0.9187977049962585
[adj_r2]:  0.9185931652103548
[coef]:  [29.50407612]
[intercept_]:  14.487714795521471
key:  Débit sève (L/j)
--------
[col_y]:  Débit sève (L/j)
[mse]:  2033741.749897235
[rmse]:  1426.0931771441988
[r2]:  0.8930362795070979
[adj_r2]:  0.8927668494806673
[coef]:  [26300.93366983]
[intercept_]:  742.8850603880696
key:  Sucre sève (%)
--------
[col_y]:  Sucre sève (%)
[mse]:  0.020679924625064783
[rmse]:  0.14380516202509833
[r2]:  0.8674769112284171
[adj_r2]:  0.8668076026992677
[coef]:  [-2.12551355  0.82989425]
[intercept_]:  2.214203423561279
key:  Transmittance produit (%)
--------
[col_y]:  Transmittance produit (%)
[mse]:  34.914870117403346
[rmse]:  5.908880614583725
[r2]:  0.752994511477405
[adj_r2]:  0.7511185204253347
[coef]:  [ 21.14054266 -17.69518087 -36.62998123]
[intercept_]:  57.102184615384616


In [53]:
### pairplots
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
i=0
for key in (dict_y_colsX.keys()):
    i+=1
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    #sns.pairplot(pd.concat([dfX, dfY], axis=0))
    #plt.figure(i)
    #print(dfX.columns.values)
    #print(dfNumerique[key].head())
    #plt.figure()
    #plt.plot(pd.concat([dfX, dfY], axis=0))
    sns.pairplot(data = dfNumerique, x_vars = dfX.columns.values , y_vars = [key], kind='reg', diag_kind=None)
    
    
    #sns.pairplot(data = reducedDf, x_vars = ['Nombre épisodes gel/dégel'], y_vars = ['Production moyenne par entaille (L)'], kind='reg', diag_kind='kde')
    #plt.show(block=False)

    

key:  Production moyenne par entaille (L)
key:  Débit sève (L/j)
key:  Sucre sève (%)
key:  Transmittance produit (%)


In [54]:
### plot regressions
#https://seaborn.pydata.org/tutorial/regression.html
i=0
for key in (dict_y_colsX.keys()):
    
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]   
    
    #plt.figure(i)
    #sns.regplot(data = dfNumerique, x = dfX, y = key)
    #plt.show(block=False)
    
    for col_x in dfX.columns.values:
        i+=1
        plt.figure(i)
        print("col_x: ", col_x)
        sns.regplot(data = dfNumerique, x = col_x, y = key)    
        plt.show(block=False)

key:  Production moyenne par entaille (L)
col_x:  Nombre épisodes gel/dégel
key:  Débit sève (L/j)
col_x:  Temps bouilloire (h)
key:  Sucre sève (%)
col_x:  Précip. tot. (mm)
col_x:  Nombre épisodes gel/dégel
key:  Transmittance produit (%)
col_x:  categSirop__Doré
col_x:  categSirop__Foncé
col_x:  categSirop__Très Foncé


In [55]:
#### END Evaluate model